In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
'''
Используется искусственно сгенерированный датасет.
Рассматривается конвеер по производству пирожных. Надо предсказать невкусные! 
'''

In [ ]:
df= pd.read_csv('df_for_EDA.csv')

In [ ]:
# Исследуем признаки

In [ ]:
df.describe(include='all')

In [ ]:
df.head()

# Date

In [ ]:
df.date.min()

In [ ]:
df.date.max()

In [ ]:
df[df.is_test==0].sort_values('date').date.iloc[[0,-1]]

In [ ]:
df[df.is_test==1].sort_values('date').date.iloc[[0,-1]]

Видим что трейн и тест разделены не по времени

In [ ]:
# поисследуем что можно сказать о частоте появления объектов
df.date = pd.to_datetime(df.date)
df.sort_values('date').date.diff().describe()

In [ ]:
# Индексы остановок
diff_date = df.sort_values('date').date.diff().dt.seconds
diff_date[diff_date > 300]

In [ ]:
# Нарисуем что было с таргетом в момент остановок

In [ ]:
df = df.sort_values('date')
df = df.reset_index()
df.rename(columns={'index': 'old_index'}, inplace=True)

In [ ]:
for v in [498, 556, 2252, 199, 183, 903]:
    new_idx = df[df.old_index == v].index[0]
    idxs = [i for i in range(new_idx - 10, new_idx + 10)]
    fig, axes = plt.subplots(figsize=(20, 5))
    plt.plot_date(df.date.loc[idxs], df.target.loc[idxs])
    plt.axvline(df.date[new_idx], color='orange')
    plt.axvline(df.date[new_idx - 1], color='orange')
    plt.title = 'v=='+str(v);

Если долго смотреть на эти графики, что можно заметить, что сразу после остановки у нескольких объектов таргет либо 1, либо неизвестен. Это можно использовать либо в качестве постобработки, либо  ввести признак - сколько времени прошло с момента остановки

# Customer

In [ ]:
fig, ax=plt.subplots(figsize=(15, 4))
plt.subplot(1, 2, 1)
sns.countplot(df[df.is_test==0].customer, order=df.customer.unique()).set_title('train');
plt.subplot(1, 2, 2)
sns.countplot(df[df.is_test==1].customer).set_title('test');

In [ ]:
fig, ax=plt.subplots(figsize=(15, 4))
plt.subplot(1, 2, 1)
sns.countplot(x='customer', hue='target', data=df[df.is_test==0], 
              order=df.customer.unique()).set_title('train');

# num_of_pink_flowers

In [ ]:
df.num_of_pink_flowers.nunique()

In [ ]:
df.num_of_pink_flowers.isnull().sum()

In [ ]:
fig, ax=plt.subplots(figsize=(15, 4))
plt.subplot(1, 2, 1)
sns.distplot(df[df.is_test==0].num_of_pink_flowers.dropna()).set_title('train');
plt.subplot(1, 2, 2)
sns.distplot(df[df.is_test==1].num_of_pink_flowers.dropna()).set_title('test');

In [ ]:
fig, ax=plt.subplots(figsize=(15, 4))
plt.subplot(1, 2, 1)
sns.countplot(x='num_of_pink_flowers', hue='target', data=df[df.is_test==0]).set_title('train');

# Flour

In [ ]:
fig, ax=plt.subplots(figsize=(15, 4))
plt.subplot(1, 2, 1)
sns.distplot(df[df.is_test==0].flour.dropna(), kde=False, bins=100).set_title('train');
plt.subplot(1, 2, 2)
sns.distplot(df[df.is_test==1].flour.dropna(), kde=False, bins=100).set_title('test');

In [ ]:
df[df.flour < 50]

In [ ]:
df[df.flour < 50].flour.describe()

In [ ]:
df[df.flour>50].flour.describe()

# temperature

In [ ]:
fig, ax=plt.subplots(figsize=(15, 4))
plt.subplot(1, 2, 1)
sns.distplot(df[df.is_test==0].temperature.dropna(), bins=40).set_title('train');
plt.subplot(1, 2, 2)
sns.distplot(df[df.is_test==1].temperature.dropna(), bins=40).set_title('test');

In [ ]:
fig, ax=plt.subplots(figsize=(10, 5))
sns.boxplot(x='target', y='temperature', data=df[df.is_test==0])

In [ ]:
fig, ax=plt.subplots(figsize=(10, 5))
sns.violinplot(x='target', y='temperature', data=df[df.is_test==0])

# type

In [ ]:
fig, ax=plt.subplots(figsize=(15, 4))
plt.subplot(1, 2, 1)
sns.countplot(df[df.is_test==0].type).set_title('train');
plt.subplot(1, 2, 2)
sns.countplot(df[df.is_test==1].type).set_title('test');

In [ ]:
fig, ax=plt.subplots(figsize=(15, 4))
plt.subplot(1, 2, 1)
sns.countplot(x='type', hue='target', data=df[df.is_test==0]).set_title('train');

In [ ]:
# не успели эту часть на семинаре

In [ ]:
# смотрим на признак
df[['date', 'type', 'target']][-50:]

In [ ]:
# Видим что изделия одного типа идут партиями. Что происходит когда одна партия сменяет другую, 

In [ ]:
# Нарисуем последние 200 точек. Будем искать индексы когда партия одного типа 
# менялась на партию другого типа
tmp = df.iloc[-201:-1, :]['type'].values != df.iloc[-200:, :]['type'].values
change_idx = df.iloc[-200:, :][tmp].index

In [ ]:
df.date[55]

In [ ]:
maper = {'A': 'blue', 'B': 'green', 'C': 'red', 'D': 'orange'}
fig, axes = plt.subplots(figsize=(20, 5))
for pcolor, gp in df.assign(color=lambda x: x.type.map(maper))[-200:].groupby('color'):
    plt.plot_date(gp['date'], gp['target'], marker='o', color=pcolor) 
for i in change_idx:
    plt.axvline(df.date[i], color='gray', ls='--')

На этом пестром графике видно, что все объекты после смены партии имеют таргет==1 или неизвестный. Те признак как давно была смена типа выглядит полезным. 

# Взаимодействия

In [ ]:
sns.catplot(x='type', y='temperature', data=df, height=5, aspect=1.5);

In [ ]:
sns.catplot(x='type', y='temperature', hue='target', data=df[df.is_test==0], height=5, aspect=1.5);